In [14]:
# Lab 11 MNIST and Deep learning CNN
import tensorflow as tf
# import matplotlib.pyplot as plt

In [15]:
from tensorflow.examples.tutorials.mnist import input_data

In [16]:
tf.set_random_seed(777)  # reproducibility

In [17]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [19]:
class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])
            # img 28x28x1 (black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                         rate=0.7, training=self.training)

            # Convolutional Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                                    padding='SAME', activation=tf.nn.relu)
            # Pooling Layer #2
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                           padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,
                                        rate=0.7, training=self.training)
            
            # Convolutional Layer #2
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],
                                    padding='SAME', activation=tf.nn.relu)
            # Pooling Layer #2
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                           padding="SAME", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,
                                        rate=0.7, training=self.training)

            # Dense Layer with Relu
            L3_flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            
            dense4 = tf.layers.dense(inputs=L3_flat,
                                     units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,
                                         rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer],
                             feed_dict={self.X: x_data, self.Y: y_data, self.training: training})

In [20]:
# To solve the problem.
# ValueError: Variable conv1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:
# refer: https://github.com/kratzert/finetune_alexnet_with_tensorflow/issues/8
tf.reset_default_graph()

In [21]:
# initialize
sess = tf.Session()

models = []
num_models = 3
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

Learning Started!


In [24]:
import numpy as np

np.zeros(len(models))

array([0., 0., 0.])

In [ ]:
# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_index, model in enumerate(models):
            c, _ = model.train(batch_xs, batch_ys)
            avg_cost_list[m_index] += c / total_batch
#             print(epoch, i, m_index, avg_cost_list)
            
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

0 0 0 [0.00071347 0.         0.        ]
0 0 1 [0.00071347 0.00092549 0.        ]
0 0 2 [0.00071347 0.00092549 0.00078154]
0 1 0 [0.0012     0.00092549 0.00078154]
0 1 1 [0.0012     0.00123263 0.00078154]
0 1 2 [0.0012     0.00123263 0.00114978]
0 2 0 [0.00155104 0.00123263 0.00114978]
0 2 1 [0.00155104 0.00181185 0.00114978]
0 2 2 [0.00155104 0.00181185 0.00175185]
0 3 0 [0.00201959 0.00181185 0.00175185]
0 3 1 [0.00201959 0.00229431 0.00175185]
0 3 2 [0.00201959 0.00229431 0.00231674]
0 4 0 [0.00281636 0.00229431 0.00231674]
0 4 1 [0.00281636 0.00287661 0.00231674]
0 4 2 [0.00281636 0.00287661 0.00293224]
0 5 0 [0.00330382 0.00287661 0.00293224]
0 5 1 [0.00330382 0.00347467 0.00293224]
0 5 2 [0.00330382 0.00347467 0.00340487]
0 6 0 [0.00424917 0.00347467 0.00340487]
0 6 1 [0.00424917 0.00426867 0.00340487]
0 6 2 [0.00424917 0.00426867 0.0041707 ]
0 7 0 [0.00496121 0.00426867 0.0041707 ]
0 7 1 [0.00496121 0.00511103 0.0041707 ]
0 7 2 [0.00496121 0.00511103 0.00486977]
0 8 0 [0.0054220

0 65 1 [0.04070455 0.04128843 0.03924351]
0 65 2 [0.04070455 0.04128843 0.03974235]
0 66 0 [0.04112123 0.04128843 0.03974235]
0 66 1 [0.04112123 0.04199013 0.03974235]
0 66 2 [0.04112123 0.04199013 0.04021909]
0 67 0 [0.04155685 0.04199013 0.04021909]
0 67 1 [0.04155685 0.04257054 0.04021909]
0 67 2 [0.04155685 0.04257054 0.04084062]
0 68 0 [0.04199737 0.04257054 0.04084062]
0 68 1 [0.04199737 0.04324628 0.04084062]
0 68 2 [0.04199737 0.04324628 0.04130789]
0 69 0 [0.04266957 0.04324628 0.04130789]
0 69 1 [0.04266957 0.04398576 0.04130789]
0 69 2 [0.04266957 0.04398576 0.04196575]
0 70 0 [0.043207   0.04398576 0.04196575]
0 70 1 [0.043207   0.04449113 0.04196575]
0 70 2 [0.043207   0.04449113 0.04245064]
0 71 0 [0.04362638 0.04449113 0.04245064]
0 71 1 [0.04362638 0.04503855 0.04245064]
0 71 2 [0.04362638 0.04503855 0.04300796]
0 72 0 [0.04415745 0.04503855 0.04300796]
0 72 1 [0.04415745 0.04557975 0.04300796]
0 72 2 [0.04415745 0.04557975 0.04347477]
0 73 0 [0.04461876 0.04557975 0.04

0 129 2 [0.07728959 0.0807269  0.07698073]
0 130 0 [0.07755545 0.0807269  0.07698073]
0 130 1 [0.07755545 0.08097163 0.07698073]
0 130 2 [0.07755545 0.08097163 0.07740629]
0 131 0 [0.07825985 0.08097163 0.07740629]
0 131 1 [0.07825985 0.08166643 0.07740629]
0 131 2 [0.07825985 0.08166643 0.07798453]
0 132 0 [0.07873534 0.08166643 0.07798453]
0 132 1 [0.07873534 0.08221759 0.07798453]
0 132 2 [0.07873534 0.08221759 0.07827385]
0 133 0 [0.0795165  0.08221759 0.07827385]
0 133 1 [0.0795165  0.0827291  0.07827385]
0 133 2 [0.0795165  0.0827291  0.07884624]
0 134 0 [0.08032713 0.0827291  0.07884624]
0 134 1 [0.08032713 0.08312309 0.07884624]
0 134 2 [0.08032713 0.08312309 0.0792859 ]
0 135 0 [0.08101189 0.08312309 0.0792859 ]
0 135 1 [0.08101189 0.08373083 0.0792859 ]
0 135 2 [0.08101189 0.08373083 0.07973034]
0 136 0 [0.08164333 0.08373083 0.07973034]
0 136 1 [0.08164333 0.08433189 0.07973034]
0 136 2 [0.08164333 0.08433189 0.08040326]
0 137 0 [0.08213211 0.08433189 0.08040326]
0 137 1 [0.

0 193 1 [0.11286977 0.11611227 0.11114986]
0 193 2 [0.11286977 0.11611227 0.11164346]
0 194 0 [0.11394937 0.11611227 0.11164346]
0 194 1 [0.11394937 0.11686403 0.11164346]
0 194 2 [0.11394937 0.11686403 0.11241113]
0 195 0 [0.11458777 0.11686403 0.11241113]
0 195 1 [0.11458777 0.11752021 0.11241113]
0 195 2 [0.11458777 0.11752021 0.11305635]
0 196 0 [0.11494675 0.11752021 0.11305635]
0 196 1 [0.11494675 0.11813506 0.11305635]
0 196 2 [0.11494675 0.11813506 0.1134911 ]
0 197 0 [0.11558094 0.11813506 0.1134911 ]
0 197 1 [0.11558094 0.11860736 0.1134911 ]
0 197 2 [0.11558094 0.11860736 0.11369409]
0 198 0 [0.11612705 0.11860736 0.11369409]
0 198 1 [0.11612705 0.11941776 0.11369409]
0 198 2 [0.11612705 0.11941776 0.1144077 ]
0 199 0 [0.11664726 0.11941776 0.1144077 ]
0 199 1 [0.11664726 0.11983535 0.1144077 ]
0 199 2 [0.11664726 0.11983535 0.11479403]
0 200 0 [0.11707689 0.11983535 0.11479403]
0 200 1 [0.11707689 0.12032527 0.11479403]
0 200 2 [0.11707689 0.12032527 0.11525171]
0 201 0 [0.

0 257 0 [0.14736663 0.15209196 0.14736725]
0 257 1 [0.14736663 0.15254284 0.14736725]
0 257 2 [0.14736663 0.15254284 0.14790859]
0 258 0 [0.14787554 0.15254284 0.14790859]
0 258 1 [0.14787554 0.15321357 0.14790859]
0 258 2 [0.14787554 0.15321357 0.14838412]
0 259 0 [0.14813746 0.15321357 0.14838412]
0 259 1 [0.14813746 0.15357928 0.14838412]
0 259 2 [0.14813746 0.15357928 0.14878797]
0 260 0 [0.14859729 0.15357928 0.14878797]
0 260 1 [0.14859729 0.15437418 0.14878797]
0 260 2 [0.14859729 0.15437418 0.14933626]
0 261 0 [0.14914135 0.15437418 0.14933626]
0 261 1 [0.14914135 0.15487884 0.14933626]
0 261 2 [0.14914135 0.15487884 0.14974997]
0 262 0 [0.14947238 0.15487884 0.14974997]
0 262 1 [0.14947238 0.15515936 0.14974997]
0 262 2 [0.14947238 0.15515936 0.15024097]
0 263 0 [0.15005513 0.15515936 0.15024097]
0 263 1 [0.15005513 0.15565671 0.15024097]
0 263 2 [0.15005513 0.15565671 0.15079778]
0 264 0 [0.1506453  0.15565671 0.15079778]
0 264 1 [0.1506453  0.1560322  0.15079778]
0 264 2 [0.

...

5 191 2 [0.05736265 0.05746152 0.05697123]
5 192 0 [0.05766331 0.05746152 0.05697123]
5 192 1 [0.05766331 0.05783264 0.05697123]
5 192 2 [0.05766331 0.05783264 0.05744645]
5 193 0 [0.05788691 0.05783264 0.05744645]
5 193 1 [0.05788691 0.05805231 0.05744645]
5 193 2 [0.05788691 0.05805231 0.05765371]
5 194 0 [0.05825148 0.05805231 0.05765371]
5 194 1 [0.05825148 0.05838185 0.05765371]
5 194 2 [0.05825148 0.05838185 0.05817442]
5 195 0 [0.05864564 0.05838185 0.05817442]
5 195 1 [0.05864564 0.05874379 0.05817442]
5 195 2 [0.05864564 0.05874379 0.05852453]
5 196 0 [0.05886403 0.05874379 0.05852453]
5 196 1 [0.05886403 0.05903754 0.05852453]
5 196 2 [0.05886403 0.05903754 0.0587486 ]
5 197 0 [0.05913054 0.05903754 0.0587486 ]
5 197 1 [0.05913054 0.05939807 0.0587486 ]
5 197 2 [0.05913054 0.05939807 0.05907736]
5 198 0 [0.05933806 0.05939807 0.05907736]
5 198 1 [0.05933806 0.05971383 0.05907736]
5 198 2 [0.05933806 0.05971383 0.05927798]
5 199 0 [0.05952149 0.05971383 0.05927798]
5 199 1 [0.

5 255 1 [0.07754127 0.07768509 0.07738133]
5 255 2 [0.07754127 0.07768509 0.07764152]
5 256 0 [0.07787788 0.07768509 0.07764152]
5 256 1 [0.07787788 0.07806775 0.07764152]
5 256 2 [0.07787788 0.07806775 0.07791968]
5 257 0 [0.07855174 0.07806775 0.07791968]
5 257 1 [0.07855174 0.0786382  0.07791968]
5 257 2 [0.07855174 0.0786382  0.07821235]
5 258 0 [0.07889368 0.0786382  0.07821235]
5 258 1 [0.07889368 0.07880486 0.07821235]
5 258 2 [0.07889368 0.07880486 0.07845079]
5 259 0 [0.07912695 0.07880486 0.07845079]
5 259 1 [0.07912695 0.07902462 0.07845079]
5 259 2 [0.07912695 0.07902462 0.07870305]
5 260 0 [0.07939106 0.07902462 0.07870305]
5 260 1 [0.07939106 0.07918672 0.07870305]
5 260 2 [0.07939106 0.07918672 0.07912136]
5 261 0 [0.07964484 0.07918672 0.07912136]
5 261 1 [0.07964484 0.07960847 0.07912136]
5 261 2 [0.07964484 0.07960847 0.07944929]
5 262 0 [0.07995196 0.07960847 0.07944929]
5 262 1 [0.07995196 0.07993827 0.07944929]
5 262 2 [0.07995196 0.07993827 0.07967674]
5 263 0 [0.

5 319 0 [0.09689836 0.09616687 0.09517712]
5 319 1 [0.09689836 0.09634629 0.09517712]
5 319 2 [0.09689836 0.09634629 0.09528398]
5 320 0 [0.09710119 0.09634629 0.09528398]
5 320 1 [0.09710119 0.09655646 0.09528398]
5 320 2 [0.09710119 0.09655646 0.09556265]
5 321 0 [0.09740727 0.09655646 0.09556265]
5 321 1 [0.09740727 0.09685744 0.09556265]
5 321 2 [0.09740727 0.09685744 0.09602471]
5 322 0 [0.0976109  0.09685744 0.09602471]
5 322 1 [0.0976109  0.09708915 0.09602471]
5 322 2 [0.0976109  0.09708915 0.09611264]
5 323 0 [0.09788235 0.09708915 0.09611264]
5 323 1 [0.09788235 0.09738574 0.09611264]
5 323 2 [0.09788235 0.09738574 0.09632331]
5 324 0 [0.0982201  0.09738574 0.09632331]
5 324 1 [0.0982201  0.09786145 0.09632331]
5 324 2 [0.0982201  0.09786145 0.09652252]
5 325 0 [0.09851006 0.09786145 0.09652252]
5 325 1 [0.09851006 0.09843142 0.09652252]
5 325 2 [0.09851006 0.09843142 0.09702098]
5 326 0 [0.09870386 0.09843142 0.09702098]
5 326 1 [0.09870386 0.09873733 0.09702098]
5 326 2 [0.

5 382 2 [0.11557878 0.11408417 0.11172333]
5 383 0 [0.11594671 0.11408417 0.11172333]
5 383 1 [0.11594671 0.11447303 0.11172333]
5 383 2 [0.11594671 0.11447303 0.11202739]
5 384 0 [0.11615101 0.11447303 0.11202739]
5 384 1 [0.11615101 0.11473293 0.11202739]
5 384 2 [0.11615101 0.11473293 0.11237583]
5 385 0 [0.11654704 0.11473293 0.11237583]
5 385 1 [0.11654704 0.11510501 0.11237583]
5 385 2 [0.11654704 0.11510501 0.11283314]
5 386 0 [0.11680564 0.11510501 0.11283314]
5 386 1 [0.11680564 0.11541887 0.11283314]
5 386 2 [0.11680564 0.11541887 0.11308479]
5 387 0 [0.11702238 0.11541887 0.11308479]
5 387 1 [0.11702238 0.11578589 0.11308479]
5 387 2 [0.11702238 0.11578589 0.11351901]
5 388 0 [0.11722517 0.11578589 0.11351901]
5 388 1 [0.11722517 0.11622009 0.11351901]
5 388 2 [0.11722517 0.11622009 0.11372035]
5 389 0 [0.11746705 0.11622009 0.11372035]
5 389 1 [0.11746705 0.11644576 0.11372035]
5 389 2 [0.11746705 0.11644576 0.11417992]
5 390 0 [0.11757881 0.11644576 0.11417992]
5 390 1 [0.

5 446 1 [0.13436293 0.13267233 0.12929551]
5 446 2 [0.13436293 0.13267233 0.12958248]
5 447 0 [0.13459206 0.13267233 0.12958248]
5 447 1 [0.13459206 0.13299996 0.12958248]
5 447 2 [0.13459206 0.13299996 0.12985074]
5 448 0 [0.13505569 0.13299996 0.12985074]
5 448 1 [0.13505569 0.13318406 0.12985074]
5 448 2 [0.13505569 0.13318406 0.13004552]
5 449 0 [0.13526489 0.13318406 0.13004552]
5 449 1 [0.13526489 0.13344212 0.13004552]
5 449 2 [0.13526489 0.13344212 0.13021499]
5 450 0 [0.13576297 0.13344212 0.13021499]
5 450 1 [0.13576297 0.1336685  0.13021499]
5 450 2 [0.13576297 0.1336685  0.13059118]
5 451 0 [0.13618012 0.1336685  0.13059118]
5 451 1 [0.13618012 0.13394091 0.13059118]
5 451 2 [0.13618012 0.13394091 0.13077716]
5 452 0 [0.13640675 0.13394091 0.13077716]
5 452 1 [0.13640675 0.13425264 0.13077716]
5 452 2 [0.13640675 0.13425264 0.13093032]
5 453 0 [0.13671175 0.13425264 0.13093032]
5 453 1 [0.13671175 0.13487103 0.13093032]
5 453 2 [0.13671175 0.13487103 0.13129112]
5 454 0 [0.

In [ ]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
print(test_size)
predictions = np.zeros([test_size, 10])
print(predictions)

for m_index, model in enumerate(models):
    print(m_index, 'Accuracy:', model.get_accuracy(mnist.test.images, mnist.test.labels))
    p = model.predict(mnist.test.images)
    predictions += p

In [ ]:
ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print('Ensemble Accuracy:', sess.run(ensemble_accuracy))